In [3]:
import os
from geographiclib.geodesic import Geodesic
import json
import cv2
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

AIRPORTS = {
    "kmdw": "#0072bb",
    "kewr": "#519872",
    "ksea": "#682D63",
    "kbos": "#ca3c25"
}

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [15]:
class TrajectoryProcessor():
    def __init__(self,  airport: str, ipath: str, opath: str, to_process: float, dpi: int ):
        self.BASE_DIR = ipath
        self.OUT_DIR =  opath
        self.AIRPORT = airport
        self.to_process = to_process
        self.color = AIRPORTS[self.AIRPORT]
        self.dpi = dpi
        
        MAP_DIR = f'{self.BASE_DIR}/maps/{self.AIRPORT}'
        LIMITS_FILE = os.path.join(MAP_DIR, 'limits.json')
        
        map_filepath = os.path.join(MAP_DIR,'bkg_map.png')
        semantic_map = cv2.imread(map_filepath)
        semantic_map = cv2.cvtColor(semantic_map, cv2.COLOR_BGR2RGB)
        self.semantic_map = cv2.resize(semantic_map, (semantic_map.shape[0]//2, semantic_map.shape[1]//2))

        with open(LIMITS_FILE, 'r') as fp:
                    reference_data = dotdict(json.load(fp))
        self.ll_limits = (reference_data.north, reference_data.east, reference_data.south, reference_data.west)
        self.reference_point = (reference_data.ref_lat,reference_data.ref_lon, reference_data.range_scale)
        
        self.TRAJECTORIES_DIR = os.path.join(self.BASE_DIR, 'raw_trajectories', airport)
        print('---- Analyzing data in ', self.TRAJECTORIES_DIR, '----')
        self.TRAJECTORY_FILES = [os.path.join(self.TRAJECTORIES_DIR, f) for f in os.listdir(self.TRAJECTORIES_DIR)]
        os.makedirs(self.OUT_DIR, exist_ok=True)
        
    def plot_trajectories(self):
        north, east, south, west = self.ll_limits
        print(f"Plotting {self.to_process*100} % of data")
        fig, map_plot = plt.subplots(1,figsize=(30,30))
        map_plot.imshow(self.semantic_map, zorder=0, extent=[west, east, south, north], alpha=1.0)
        #Plot all XY
        for i in tqdm(range(0, int(len(self.TRAJECTORY_FILES)*self.to_process))):
            test_file = self.TRAJECTORY_FILES[i]
            data = pd.read_csv(test_file)
            data.dropna(subset = ['Lat', 'Lon'])
            data = data[:][data['Type'] == 0]
            lon, lat = np.array(data.Lon.values), np.array(data.Lat.values)
            map_plot.scatter(lon, lat, c = self.color, s= 0.3 , alpha= 0.15)
            map_plot.tick_params(color='gray', labelcolor='gray')
            for spine in map_plot.spines.values():
                spine.set_edgecolor('white')
        plt.tight_layout()
        plt.xticks([])
        plt.yticks([])
        print('Saving...')
        plt.savefig(f'{self.OUT_DIR}/{self.AIRPORT}.png', dpi=self.dpi, bbox_inches='tight')
        plt.close()
        # plt.show(block = False)
        
    def heat_map(self):
        # # fig, map_plot = plt.subplots(1,figsize=(30,30))
        # map_plot.imshow(semantic_map, zorder=0, extent=[west, east, south, north], alpha=1.0)
        # Plot all XY
        print(f"Plotting {self.to_process*100} % of data")
        Lon, Lat = [], []
        for i in tqdm(range(0, int(len(self.TRAJECTORY_FILES )*self.to_process))):
            test_file = self.TRAJECTORY_FILES [i]
            data = pd.read_csv(test_file)
            data.dropna(subset = ['Lat', 'Lon'])
            data = data[:][data['Type'] == 0]
            lon, lat = np.array(data.Lon.values), np.array(data.Lat.values)
            Lon.append(lon)
            Lat.append(lat)
        Lon = np.concatenate(Lon)
        Lat = np.concatenate(Lat)
        
        freq, xedges, yedges = np.histogram2d(x = lon, y = lat, bins = 10000, density= False)
        freq = np.clip(freq, a_min= 0,a_max= 5)
        freq = freq.T
        x,y = np.meshgrid(xedges,yedges)
        plt.pcolormesh(x, y, freq)
        plt.axis('equal')
        plt.savefig(f'all_trajectories.png', dpi=self.dpi)

In [21]:

processor = TrajectoryProcessor(
    airport='kewr', 
    ipath='../../datasets/swim', 
    opath='./out/maps', 
    to_process=0.5,
    dpi=100
)
processor.plot_trajectories()

---- Analyzing data in  ../../datasets/swim/raw_trajectories/kewr ----
Plotting 50.0 % of data


100%|██████████| 698/698 [00:43<00:00, 16.09it/s]


Saving...
